In [5]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model
import time

# MediaPipe 세팅
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True)
drawing_spec = mp.solutions.drawing_utils.DrawingSpec(thickness=1, circle_radius=1)

# 눈 랜드마크 인덱스
left_eye_indices = [33, 160, 158, 157, 154, 153, 145, 144, 163, 7, 173, 246, 130, 133, 159]
right_eye_indices = [362, 385, 387, 388, 390, 373, 374, 380, 382, 263, 466, 359, 386, 384, 362]

# 모델 불러오기 (MobileNet)
model = load_model("C:/train/eye_model_fold_1.h5")

# MobileNet 입력 크기
input_size = (224, 224)

# 눈이 감긴 시간을 추적할 변수
eye_closed_time = 0
closed_start_time = None
eye_closed_count = 0  # 눈 감은 횟수

# 번쩍거리기 효과를 위한 타이머
blink_start_time = None
blink_display_time = 3  # 번쩍거리는 시간 (초)
blink_flash = False

# 카메라 켜기
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # 이미지를 RGB로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # 얼굴 랜드마크 추출
    results = face_mesh.process(rgb_frame)
    
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            h, w, _ = frame.shape
            
            # 왼쪽, 오른쪽 눈 좌표 계산
            left_eye_coords = np.array([(int(face_landmarks.landmark[i].x * w), int(face_landmarks.landmark[i].y * h)) for i in left_eye_indices])
            right_eye_coords = np.array([(int(face_landmarks.landmark[i].x * w), int(face_landmarks.landmark[i].y * h)) for i in right_eye_indices])

            # 눈 주위를 감싸는 직사각형 영역 얻기
            left_eye_rect = cv2.boundingRect(left_eye_coords)
            right_eye_rect = cv2.boundingRect(right_eye_coords)

            # 눈보다 약간 큰 사이즈로 영역 확장
            eye_margin = 5
            left_eye_x, left_eye_y, left_eye_w, left_eye_h = left_eye_rect
            right_eye_x, right_eye_y, right_eye_w, right_eye_h = right_eye_rect

            left_eye_roi = frame[left_eye_y-eye_margin:left_eye_y+left_eye_h+eye_margin, left_eye_x-eye_margin:left_eye_x+left_eye_w+eye_margin]
            right_eye_roi = frame[right_eye_y-eye_margin:right_eye_y+right_eye_h+eye_margin, right_eye_x-eye_margin:right_eye_x+right_eye_w+eye_margin]

            # 눈을 MobileNet에 맞는 크기로 리사이징 (224x224)
            left_eye_resized = cv2.resize(left_eye_roi, input_size)
            right_eye_resized = cv2.resize(right_eye_roi, input_size)
            
            # MobileNet 입력에 맞게 전처리 (정규화 및 배치 차원 추가)
            left_eye_input = np.expand_dims(left_eye_resized / 255.0, axis=0)
            right_eye_input = np.expand_dims(right_eye_resized / 255.0, axis=0)
            
            # 모델로 예측 (0: 눈 감음, 1: 눈 뜸)
            left_eye_pred = model.predict(left_eye_input)
            right_eye_pred = model.predict(right_eye_input)
            
            # 두 눈 모두 감은 경우를 확인
            if left_eye_pred < 0.5 and right_eye_pred < 0.5:
                if closed_start_time is None:
                    closed_start_time = time.time()
                else:
                    eye_closed_time = time.time() - closed_start_time
                
                # 2초 이상 눈 감으면 횟수 증가
                if eye_closed_time >= 2:
                    print("눈을 감음")
                    eye_closed_count += 1
                    closed_start_time = None  # 다시 카운트 시작하기 위해 초기화
            else:
                # 눈이 뜬 경우 시간 초기화
                closed_start_time = None
                eye_closed_time = 0
            
            # 눈 영역에 직사각형 그리기
            cv2.rectangle(frame, (left_eye_x-eye_margin, left_eye_y-eye_margin), 
                          (left_eye_x+left_eye_w+eye_margin, left_eye_y+left_eye_h+eye_margin), (0, 255, 0), 2)
            cv2.rectangle(frame, (right_eye_x-eye_margin, right_eye_y-eye_margin), 
                          (right_eye_x+right_eye_w+eye_margin, right_eye_y+right_eye_h+eye_margin), (0, 255, 0), 2)

    # 눈 감은 횟수가 3번 이상이면 "SLEEP!!" 표시
    if eye_closed_count >= 3:
        if blink_start_time is None:
            blink_start_time = time.time()  # 번쩍거리기 시작 시간 설정

        # 3초 동안 번쩍거리면서 "SLEEP!!" 표시
        if time.time() - blink_start_time <= blink_display_time:
            if int(time.time() * 10) % 2 == 0:  # 0.1초마다 번쩍거리기
                cv2.putText(frame, "SLEEP!!", (50, h // 2), cv2.FONT_HERSHEY_SIMPLEX, 4, (0, 0, 255), 10, cv2.LINE_AA)
        else:
            # 3초가 지나면 화면을 빨간색으로 채움
            frame[:] = (0, 0, 255)

    # 결과 출력
    cv2.imshow('Eye Detection', frame)
    
    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 43ms/step
